In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import models, transforms
from tqdm import tqdm
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ResNet50 and remove final classification layer
resnet = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
resnet.fc = torch.nn.Identity()
resnet = resnet.to(device)
resnet.eval()

print("Model ready on:", device)


Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 214MB/s]


Model ready on: cuda


In [2]:
def get_image_path(article_id):
    article_str = str(article_id).zfill(10)
    folder = article_str[:3]
    return f"/kaggle/input/competitions/h-and-m-personalized-fashion-recommendations/images/{folder}/{article_str}.jpg"


In [3]:
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os

class FashionDataset(Dataset):
    def __init__(self, image_ids, image_folder, transform):
        self.image_ids = image_ids
        self.image_folder = image_folder
        self.transform = transform
        
    def __len__(self):
        return len(self.image_ids)
    
    def __getitem__(self, idx):
        article_id = self.image_ids[idx]
        img_path = os.path.join(self.image_folder, f"{article_id}.jpg")
        
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        
        return image, article_id

In [4]:
import pandas as pd

articles = pd.read_csv(
    "/kaggle/input/competitions/h-and-m-personalized-fashion-recommendations/articles.csv"
)

print("Articles shape:", articles.shape)


Articles shape: (105542, 25)


In [5]:
import os

def get_image_path(article_id):
    article_str = str(article_id).zfill(10)
    folder = article_str[:3]
    return f"/kaggle/input/competitions/h-and-m-personalized-fashion-recommendations/images/{folder}/{article_str}.jpg"


In [6]:
import torchvision.transforms as transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [7]:
from torch.utils.data import Dataset
from PIL import Image
import os

class FashionDataset(Dataset):
    def __init__(self, article_ids, transform):
        self.article_ids = article_ids
        self.transform = transform
        
    def __len__(self):
        return len(self.article_ids)
    
    def __getitem__(self, idx):
        article_id = self.article_ids[idx]
        # Build image path
        article_str = str(article_id).zfill(10)
        folder = article_str[:3]
        img_path = f"/kaggle/input/competitions/h-and-m-personalized-fashion-recommendations/images/{folder}/{article_str}.jpg"
        
        # Skip missing images
        if not os.path.exists(img_path):
            # Return None, the DataLoader collate_fn will skip it
            return None
        
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        return image, article_id


In [8]:
def collate_fn(batch):
    batch = [b for b in batch if b is not None]
    if len(batch) == 0:
        return None
    images, ids = zip(*batch)
    images = torch.stack(images, dim=0)
    return images, ids


In [9]:
# Correct order: first define valid_article_ids
article_ids = articles["article_id"].unique().tolist()
valid_article_ids = [aid for aid in article_ids if os.path.exists(get_image_path(aid))]
print(f"Total articles: {len(article_ids)}")
print(f"Articles with images: {len(valid_article_ids)}")


Total articles: 105542
Articles with images: 105100


In [10]:
# DataLoader using only articles with images
dataset = FashionDataset(valid_article_ids, transform)
loader = DataLoader(
    dataset,
    batch_size=64,
    shuffle=False,
    num_workers=2,
    collate_fn=collate_fn
)


In [11]:
import torch
from tqdm import tqdm
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet.eval()
resnet.to(device)

item_embeddings = {}

with torch.no_grad():
    for batch in tqdm(loader):
        if batch is None:
            continue
        images, ids = batch
        images = images.to(device)
        embeddings = resnet(images).cpu().numpy()
        for i, article_id in enumerate(ids):
            item_embeddings[int(article_id)] = embeddings[i]


100%|██████████| 1643/1643 [32:34<00:00,  1.19s/it]


In [12]:
import pickle

with open("/kaggle/working/image_embeddings.pkl", "wb") as f:
    pickle.dump(item_embeddings, f)

print("Saved successfully.")

Saved successfully.


In [13]:
from sklearn.metrics.pairwise import cosine_similarity

all_items = list(item_embeddings.keys())
all_vectors = np.array([item_embeddings[i] for i in all_items])

def recommend_similar_items(query_item_id, top_k=12):
    
    if query_item_id not in item_embeddings:
        return []
    
    query_vector = item_embeddings[query_item_id].reshape(1, -1)
    similarities = cosine_similarity(query_vector, all_vectors)[0]
    
    top_indices = similarities.argsort()[-top_k-1:][::-1]
    
    recommended = [
        all_items[i]
        for i in top_indices
        if all_items[i] != query_item_id
    ]
    
    return recommended[:top_k]

In [14]:
recommend_similar_items(article_ids[0])

[626366003,
 626366002,
 767834001,
 717464001,
 733814001,
 572799008,
 824999001,
 538699001,
 647522002,
 800869001,
 868680001,
 651465001]

# Phase 6 — Image-Based Recommendation

In this phase, we build a **visual recommender system** for fashion items using product images.  
The main goals are:

- Extract **image embeddings** using a pretrained CNN (ResNet50).  
- Store **item-level visual vectors** for all products.  
- Recommend **visually similar items** based on cosine similarity.  
- Prepare for the **Hybrid Recommender System** (Phase 7).

---

##  GPU Setup

For efficient computation, we use Kaggle GPU:

```python
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Available:", torch.cuda.is_available())
